<a href="https://colab.research.google.com/github/jay6-dev/conversational-health-risk-analyzer/blob/main/Conversational_health_risk_analyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
"""
Healthcare Conversation Risk Assessment Tool
=============================================
Time Spent: 1:30 hrs

This notebook analyzes WhatsApp health conversations to:
1. Assess HIV acquisition risk
2. Evaluate mental health disorder risk
3. Provide SA NDOH-aligned recommendations

Author: Risk Assessment Tool
Date: 24 November 2025
"""

# IMPORTS AND SETUP


import re
import pandas as pd
from datetime import datetime
from typing import Dict, List, Tuple
import json


# LOADING DATA AND PREPROCESSING

def parse_whatsapp_conversations(file_path: str) -> List[Dict]:
    """
    Parse WhatsApp-style conversation logs into structured format.

    Args:
        file_path: Path to the conversation text file

    Returns:
        List of conversation dictionaries
    """
    with open(file_path, 'r', encoding='utf-8') as f:
        content = f.read()

    # Split by conversation separator
    conversations = content.split('========== Conversation ==========')

    parsed_conversations = []
    for idx, conv in enumerate(conversations):
        if not conv.strip():
            continue

        messages = []
        lines = conv.strip().split('\n')

        for line in lines:
            # Parse WhatsApp format: [date, time] Speaker: Message
            match = re.match(r'\[(.*?)\] (User|AI): (.*)', line)
            if match:
                timestamp, speaker, message = match.groups()
                messages.append({
                    'timestamp': timestamp,
                    'speaker': speaker,
                    'message': message.strip()
                })

        if messages:
            parsed_conversations.append({
                'conversation_id': idx + 1,
                'messages': messages,
                'user_messages': [m['message'] for m in messages if m['speaker'] == 'User'],
                'ai_messages': [m['message'] for m in messages if m['speaker'] == 'AI']
            })

    return parsed_conversations


# ============================================================================
# HIV RISK ASSESSMENT ENGINE
# ============================================================================

class HIVRiskAssessor:
    """
    Assesses HIV acquisition risk based on conversational patterns and keywords.
    Aligned with South African NDOH HIV prevention guidelines.
    """

    # Risk indicators based on SA NDOH guidelines
    HIGH_RISK_KEYWORDS = {
        'sexual': ['partner', 'unprotected', 'condom', 'sex', 'sexual'],
        'symptoms': ['discharge', 'sores', 'burning', 'pain', 'genital', 'sti', 'std'],
        'exposure': ['exposed', 'risky', 'unsafe', 'multiple partners'],
        'testing': ['test', 'hiv', 'positive', 'negative', 'status']
    }

    BEHAVIORAL_PATTERNS = {
        'concern_about_partner': ['partner', 'concerned about', 'should i be concerned'],
        'symptom_persistence': ['started', 'days ago', 'hasn\'t', 'go away', 'still'],
        'anxiety_indicators': ['worried', 'stressed', 'scared', 'anxious', 'afraid']
    }

    def assess_risk(self, conversation: Dict) -> Dict:
        """
        Calculate HIV risk score and identify risk factors.

        Returns:
            Dictionary with risk score (0-100) and detailed assessment
        """
        user_messages = ' '.join(conversation['user_messages']).lower()
        ai_messages = ' '.join(conversation['ai_messages']).lower()

        risk_score = 0
        risk_factors = []

        # Check for high-risk keywords (40 points max)
        for category, keywords in self.HIGH_RISK_KEYWORDS.items():
            matches = sum(1 for kw in keywords if kw in user_messages)
            if matches > 0:
                category_score = min(matches * 10, 15)
                risk_score += category_score
                risk_factors.append(f"{category.replace('_', ' ').title()}: {matches} indicators")

        # Check behavioral patterns (30 points max)
        for pattern, keywords in self.BEHAVIORAL_PATTERNS.items():
            matches = sum(1 for kw in keywords if kw in user_messages)
            if matches > 0:
                pattern_score = min(matches * 5, 10)
                risk_score += pattern_score
                risk_factors.append(f"{pattern.replace('_', ' ').title()}: detected")

        # Persistence and urgency analysis (30 points max)
        if 'urgent' in user_messages or 'emergency' in user_messages:
            risk_score += 15
            risk_factors.append("Urgency expressed")

        if 'days' in user_messages or 'weeks' in user_messages:
            risk_score += 10
            risk_factors.append("Duration mentioned")

        if 'need to see' in user_messages or 'see someone' in user_messages:
            risk_score += 5
            risk_factors.append("Seeking medical attention")

        # Normalize score to 0-100
        risk_score = min(risk_score, 100)

        # Classify risk level
        if risk_score >= 70:
            risk_level = "HIGH"
        elif risk_score >= 40:
            risk_level = "MODERATE"
        else:
            risk_level = "LOW"

        return {
            'risk_score': risk_score,
            'risk_level': risk_level,
            'risk_factors': risk_factors,
            'conversation_length': len(conversation['user_messages'])
        }


# ============================================================================
# MENTAL HEALTH RISK ASSESSMENT ENGINE
# ============================================================================

class MentalHealthAssessor:
    """
    Assesses mental health disorder risk using validated screening indicators.
    Based on PHQ-9, GAD-7, and SA mental health protocols.
    """

    # Depression indicators (PHQ-9 aligned)
    DEPRESSION_INDICATORS = {
        'mood': ['sad', 'depressed', 'down', 'hopeless', 'empty', 'worthless'],
        'anhedonia': ['no interest', 'don\'t enjoy', 'no pleasure', 'unmotivated'],
        'sleep': ['can\'t sleep', 'insomnia', 'sleeping too much', 'tired'],
        'appetite': ['no appetite', 'eating too much', 'weight loss', 'weight gain'],
        'concentration': ['can\'t focus', 'can\'t concentrate', 'forgetful'],
        'suicidal': ['hurt myself', 'end it', 'better off dead', 'suicide']
    }

    # Anxiety indicators (GAD-7 aligned)
    ANXIETY_INDICATORS = {
        'worry': ['worried', 'worrying', 'anxious', 'nervous', 'stressed'],
        'restlessness': ['restless', 'on edge', 'can\'t relax', 'tense'],
        'irritability': ['irritable', 'angry', 'frustrated', 'annoyed'],
        'fear': ['afraid', 'scared', 'panic', 'terrified', 'dread'],
        'physical': ['heart racing', 'sweating', 'trembling', 'dizzy']
    }

    # Stress and coping indicators
    STRESS_INDICATORS = ['stressed', 'overwhelmed', 'can\'t cope', 'breaking down']

    def assess_risk(self, conversation: Dict) -> Dict:
        """
        Calculate mental health risk score and identify concerns.

        Returns:
            Dictionary with risk score (0-100) and detailed assessment
        """
        user_messages = ' '.join(conversation['user_messages']).lower()

        depression_score = 0
        anxiety_score = 0
        stress_score = 0
        concerns = []

        # Depression assessment (40 points max)
        for category, keywords in self.DEPRESSION_INDICATORS.items():
            matches = sum(1 for kw in keywords if kw in user_messages)
            if matches > 0:
                if category == 'suicidal':
                    depression_score += 20  # Critical indicator
                    concerns.append(f"CRITICAL: {category.title()} ideation detected")
                else:
                    depression_score += min(matches * 5, 10)
                    concerns.append(f"Depression: {category.title()} symptoms")

        # Anxiety assessment (40 points max)
        for category, keywords in self.ANXIETY_INDICATORS.items():
            matches = sum(1 for kw in keywords if kw in user_messages)
            if matches > 0:
                anxiety_score += min(matches * 5, 10)
                concerns.append(f"Anxiety: {category.title()} symptoms")

        # Stress assessment (20 points max)
        stress_matches = sum(1 for kw in self.STRESS_INDICATORS if kw in user_messages)
        if stress_matches > 0:
            stress_score = min(stress_matches * 10, 20)
            concerns.append(f"Stress: {stress_matches} indicators")

        # Calculate overall risk score
        risk_score = min(depression_score + anxiety_score + stress_score, 100)

        # Classify risk level
        if risk_score >= 70 or 'suicidal' in str(concerns).lower():
            risk_level = "HIGH"
        elif risk_score >= 40:
            risk_level = "MODERATE"
        else:
            risk_level = "LOW"

        return {
            'risk_score': risk_score,
            'risk_level': risk_level,
            'depression_score': depression_score,
            'anxiety_score': anxiety_score,
            'stress_score': stress_score,
            'concerns': concerns,
            'conversation_length': len(conversation['user_messages'])
        }


# ============================================================================
# SA NDOH TREATMENT RECOMMENDATIONS ENGINE
# ============================================================================

class NDOHTreatmentPlanner:
    """
    Generates treatment recommendations aligned with SA National Department
    of Health (NDOH) guidelines for HIV and mental health.
    """

    def generate_hiv_recommendations(self, risk_assessment: Dict) -> Dict:
        """
        Generate HIV-specific recommendations per SA NDOH guidelines.
        """
        risk_level = risk_assessment['risk_level']
        risk_score = risk_assessment['risk_score']

        if risk_level == "HIGH":
            recommendations = {
                'urgency': 'IMMEDIATE',
                'actions': [
                    'Visit nearest clinic or health facility within 24 hours',
                    'Request HIV rapid test (available free at public clinics)',
                    'If potential exposure within 72 hours, inquire about Post-Exposure Prophylaxis (PEP)',
                    'Discuss Pre-Exposure Prophylaxis (PrEP) eligibility with healthcare provider',
                    'Get screened for other STIs'
                ],
                'resources': [
                    'AIDS Helpline: 0800 012 322 (toll-free, 24/7)',
                    'NDOH HIV Counselling: 0800 111 321',
                    'Find nearest clinic: https://www.health.gov.za or dial *134*1234#'
                ],
                'follow_up': 'Weekly clinic visits until status confirmed and treatment initiated if needed'
            }
        elif risk_level == "MODERATE":
            recommendations = {
                'urgency': 'WITHIN 1 WEEK',
                'actions': [
                    'Schedule HIV test at local clinic (free service)',
                    'Attend safer sex counselling session',
                    'Learn about PrEP availability for ongoing protection',
                    'Consider STI screening',
                    'Receive condom provision and education'
                ],
                'resources': [
                    'AIDS Helpline: 0800 012 322',
                    'Local clinic finder: *134*1234#',
                    'LoveLife for youth support: 0800 121 900'
                ],
                'follow_up': 'Monthly check-ins, quarterly HIV testing if risk continues'
            }
        else:
            recommendations = {
                'urgency': 'ROUTINE',
                'actions': [
                    'Routine HIV testing every 6-12 months (recommended for all adults)',
                    'Access free condoms at clinics and public health facilities',
                    'Attend health education sessions when available',
                    'Know your status - testing is confidential and free'
                ],
                'resources': [
                    'General health info: 0800 111 321',
                    'Clinic finder: *134*1234#'
                ],
                'follow_up': 'Annual health check-ups'
            }

        # Add treatment plan if high risk
        if risk_level == "HIGH":
            recommendations['treatment_plan'] = {
                'immediate': [
                    'HIV testing (rapid test results in 20 minutes)',
                    'Clinical assessment by nurse or doctor',
                    'Baseline tests: CD4 count, viral load if positive'
                ],
                'if_positive': [
                    'Same-day ART initiation (per SA Universal Test & Treat policy)',
                    'Adherence counselling',
                    'Partner notification services',
                    'Regular viral load monitoring (3 months, 6 months, then 6-monthly)'
                ],
                'prevention': [
                    'PrEP initiation if negative but high ongoing risk',
                    'Condom provision and education',
                    'STI screening and treatment'
                ]
            }

        return recommendations

    def generate_mental_health_recommendations(self, risk_assessment: Dict) -> Dict:
        """
        Generate mental health recommendations per SA NDOH guidelines.
        """
        risk_level = risk_assessment['risk_level']
        concerns = risk_assessment.get('concerns', [])

        # Check for crisis indicators
        is_crisis = any('CRITICAL' in str(c) or 'suicidal' in str(c).lower() for c in concerns)

        if is_crisis or risk_level == "HIGH":
            recommendations = {
                'urgency': 'EMERGENCY' if is_crisis else 'URGENT (24-48 hours)',
                'actions': [
                    'IMMEDIATE: Call 24/7 Crisis Line - 0800 567 567 (SADAG)' if is_crisis else 'Contact crisis support within 24 hours',
                    'Visit nearest hospital emergency department if in immediate danger',
                    'Schedule urgent psychiatric assessment at clinic or district hospital',
                    'Do not remain alone - contact trusted friend or family member',
                    'Request referral to psychologist/psychiatrist (available at public facilities)'
                ],
                'resources': [
                    'SADAG 24/7 Crisis Line: 0800 567 567 (toll-free)',
                    'Suicide Crisis Line: 0800 567 567',
                    'LifeLine: 0861 322 322',
                    'Mental Health Helpline: 0800 456 789',
                    'WhatsApp Support: 076 882 2775 (SADAG)'
                ],
                'follow_up': 'Daily contact for first week, then twice weekly for month 1'
            }
        elif risk_level == "MODERATE":
            recommendations = {
                'urgency': 'WITHIN 1-2 WEEKS',
                'actions': [
                    'Book appointment with clinic nurse for mental health screening',
                    'Request referral to counsellor (free at public clinics)',
                    'Join support group if available in community',
                    'Practice self-care: regular sleep, exercise, healthy eating',
                    'Reduce alcohol and substance use if applicable'
                ],
                'resources': [
                    'SADAG Mental Health Line: 011 234 4837',
                    'Clinic finder: *134*1234#',
                    'Mental Health Matters: 0800 456 789',
                    'Free counselling at clinics and community health centers'
                ],
                'follow_up': 'Fortnightly counselling sessions for 3 months'
            }
        else:
            recommendations = {
                'urgency': 'ROUTINE',
                'actions': [
                    'Continue monitoring mood and stress levels',
                    'Practice stress management techniques',
                    'Maintain social connections',
                    'Access wellness programs at clinic if needed',
                    'Annual mental health check during routine clinic visits'
                ],
                'resources': [
                    'SADAG Information Line: 011 234 4837',
                    'Mental Health SA: www.mentalhealthsa.org',
                    'Free clinic services: *134*1234#'
                ],
                'follow_up': 'As needed, routine annual screening'
            }

        # Add treatment plan for moderate-high risk
        if risk_level in ["HIGH", "MODERATE"]:
            recommendations['treatment_plan'] = {
                'psychological': [
                    'Individual counselling (CBT preferred, available at facilities)',
                    'Group therapy if available',
                    'Psychoeducation sessions',
                    'Stress management training'
                ],
                'pharmacological': [
                    'Psychiatric assessment for medication if needed',
                    'First-line antidepressants (available free: fluoxetine, amitriptyline)',
                    'Anxiolytics only if severe anxiety (short-term use)',
                    'Regular medication review and monitoring'
                ] if risk_level == "HIGH" else ['May be considered after assessment'],
                'social': [
                    'Social work referral for psychosocial support',
                    'Disability grant assessment if severe impairment',
                    'Community support group linkage',
                    'Family counselling if appropriate'
                ]
            }

        return recommendations


# ============================================================================
# MAIN ANALYSIS FUNCTION
# ============================================================================

def analyze_conversation(conversation: Dict) -> Dict:
    """
    Comprehensive analysis of a single conversation.

    Returns:
        Complete risk assessment and recommendations
    """
    hiv_assessor = HIVRiskAssessor()
    mh_assessor = MentalHealthAssessor()
    planner = NDOHTreatmentPlanner()

    # Perform assessments
    hiv_risk = hiv_assessor.assess_risk(conversation)
    mh_risk = mh_assessor.assess_risk(conversation)

    # Generate recommendations
    hiv_recs = planner.generate_hiv_recommendations(hiv_risk)
    mh_recs = planner.generate_mental_health_recommendations(mh_risk)

    return {
        'conversation_id': conversation['conversation_id'],
        'hiv_assessment': hiv_risk,
        'mental_health_assessment': mh_risk,
        'hiv_recommendations': hiv_recs,
        'mental_health_recommendations': mh_recs,
        'overall_priority': max(
            1 if hiv_risk['risk_level'] == 'HIGH' or mh_risk['risk_level'] == 'HIGH' else 0,
            0.5 if hiv_risk['risk_level'] == 'MODERATE' or mh_risk['risk_level'] == 'MODERATE' else 0
        )
    }


def analyze_all_conversations(file_path: str) -> pd.DataFrame:
    """
    Analyze all conversations and create summary DataFrame.
    """
    conversations = parse_whatsapp_conversations(file_path)
    results = []

    for conv in conversations:
        analysis = analyze_conversation(conv)
        results.append({
            'Conversation_ID': analysis['conversation_id'],
            'HIV_Risk_Score': analysis['hiv_assessment']['risk_score'],
            'HIV_Risk_Level': analysis['hiv_assessment']['risk_level'],
            'MH_Risk_Score': analysis['mental_health_assessment']['risk_score'],
            'MH_Risk_Level': analysis['mental_health_assessment']['risk_level'],
            'Priority': analysis['overall_priority'],
            'Full_Analysis': analysis
        })

    return pd.DataFrame(results)


# ============================================================================
# REPORTING AND VISUALIZATION
# ============================================================================

def generate_detailed_report(analysis: Dict) -> str:
    """
    Generate a detailed clinical report for a single conversation.
    """
    report = f"""
{'='*80}
HEALTHCARE RISK ASSESSMENT REPORT
{'='*80}

Conversation ID: {analysis['conversation_id']}
Date Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

{'='*80}
HIV ACQUISITION RISK ASSESSMENT
{'='*80}

Risk Score: {analysis['hiv_assessment']['risk_score']}/100
Risk Level: {analysis['hiv_assessment']['risk_level']}

Risk Factors Identified:
"""
    for factor in analysis['hiv_assessment']['risk_factors']:
        report += f"  • {factor}\n"

    report += f"""
{'-'*80}
RECOMMENDATIONS & TREATMENT PLAN (HIV)
{'-'*80}

Urgency: {analysis['hiv_recommendations']['urgency']}

Immediate Actions:
"""
    for action in analysis['hiv_recommendations']['actions']:
        report += f"  {len(analysis['hiv_recommendations']['actions'])}. {action}\n"

    report += "\nAvailable Resources:\n"
    for resource in analysis['hiv_recommendations']['resources']:
        report += f"  • {resource}\n"

    if 'treatment_plan' in analysis['hiv_recommendations']:
        report += "\nDetailed Treatment Plan:\n"
        for phase, items in analysis['hiv_recommendations']['treatment_plan'].items():
            report += f"\n  {phase.upper().replace('_', ' ')}:\n"
            for item in items:
                report += f"    • {item}\n"

    report += f"""
{'='*80}
MENTAL HEALTH RISK ASSESSMENT
{'='*80}

Overall Risk Score: {analysis['mental_health_assessment']['risk_score']}/100
Risk Level: {analysis['mental_health_assessment']['risk_level']}

Component Scores:
  • Depression Indicators: {analysis['mental_health_assessment']['depression_score']}/40
  • Anxiety Indicators: {analysis['mental_health_assessment']['anxiety_score']}/40
  • Stress Indicators: {analysis['mental_health_assessment']['stress_score']}/20

Concerns Identified:
"""
    for concern in analysis['mental_health_assessment']['concerns']:
        report += f"  • {concern}\n"

    report += f"""
{'-'*80}
RECOMMENDATIONS & TREATMENT PLAN (MENTAL HEALTH)
{'-'*80}

Urgency: {analysis['mental_health_recommendations']['urgency']}

Immediate Actions:
"""
    for i, action in enumerate(analysis['mental_health_recommendations']['actions'], 1):
        report += f"  {i}. {action}\n"

    report += "\nSupport Resources:\n"
    for resource in analysis['mental_health_recommendations']['resources']:
        report += f"  • {resource}\n"

    if 'treatment_plan' in analysis['mental_health_recommendations']:
        report += "\nDetailed Treatment Plan:\n"
        for modality, interventions in analysis['mental_health_recommendations']['treatment_plan'].items():
            report += f"\n  {modality.upper().replace('_', ' ')} INTERVENTIONS:\n"
            for intervention in interventions:
                report += f"    • {intervention}\n"

    report += f"""
{'='*80}
FOLLOW-UP CARE
{'='*80}

HIV Follow-up: {analysis['hiv_recommendations']['follow_up']}
Mental Health Follow-up: {analysis['mental_health_recommendations']['follow_up']}

{'='*80}
END OF REPORT
{'='*80}
"""

    return report


# ============================================================================
# EXECUTION AND RESULTS
# ============================================================================

if __name__ == "__main__":
    print("Healthcare Risk Assessment System")
    print("=" * 80)
    print(f"Analysis Started: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print()

    # File path - REQUIRED: Place the data file in the same directory as this notebook
    file_path = '/health_ai_whatsapp_100_conversations_long.txt'

    # Verify file exists
    import os
    if not os.path.exists(file_path):
        print(f"ERROR: File not found at '{file_path}'")
        print("Please ensure 'health_ai_whatsapp_100_conversations_long.txt' is in the same directory.")
        exit(1)

    print(f"Loading conversations from: {file_path}")

    # Analyze all conversations
    results_df = analyze_all_conversations(file_path)

    # Display summary statistics
    print("\n" + "=" * 80)
    print("SUMMARY STATISTICS")
    print("=" * 80)
    print(f"Total Conversations Analyzed: {len(results_df)}")
    print()

    print("HIV Risk Distribution:")
    print(results_df['HIV_Risk_Level'].value_counts())
    print(f"Average HIV Risk Score: {results_df['HIV_Risk_Score'].mean():.1f}/100")
    print()

    print("Mental Health Risk Distribution:")
    print(results_df['MH_Risk_Level'].value_counts())
    print(f"Average MH Risk Score: {results_df['MH_Risk_Score'].mean():.1f}/100")
    print()

    print("Priority Cases:")
    high_priority = results_df[results_df['Priority'] == 1]
    print(f"  High Priority (Immediate Attention): {len(high_priority)} conversations")

    # Display detailed reports for high-priority cases
    print("\n" + "=" * 80)
    print("DETAILED REPORTS FOR HIGH-PRIORITY CASES")
    print("=" * 80)

    for idx, row in high_priority.head(3).iterrows():  # Show first 3 high-priority cases
        detailed_report = generate_detailed_report(row['Full_Analysis'])
        print(detailed_report)

    # Save results
    results_df_export = results_df.drop('Full_Analysis', axis=1)
    print(f"\nResults saved to: risk_assessment_results.csv")

    print(f"\nAnalysis Completed: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print("=" * 80)

    # Display the DataFrame
    print("\nSummary Table:")
    print(results_df_export.to_string(index=False))

Healthcare Risk Assessment System
Analysis Started: 2025-11-24 09:31:09

Loading conversations from: /health_ai_whatsapp_100_conversations_long.txt

SUMMARY STATISTICS
Total Conversations Analyzed: 100

HIV Risk Distribution:
HIV_Risk_Level
HIGH        72
MODERATE    28
Name: count, dtype: int64
Average HIV Risk Score: 69.3/100

Mental Health Risk Distribution:
MH_Risk_Level
LOW    100
Name: count, dtype: int64
Average MH Risk Score: 15.0/100

Priority Cases:
  High Priority (Immediate Attention): 72 conversations

DETAILED REPORTS FOR HIGH-PRIORITY CASES

HEALTHCARE RISK ASSESSMENT REPORT

Conversation ID: 1
Date Generated: 2025-11-24 09:31:09

HIV ACQUISITION RISK ASSESSMENT

Risk Score: 75/100
Risk Level: HIGH

Risk Factors Identified:
  • Sexual: 1 indicators
  • Testing: 1 indicators
  • Concern About Partner: detected
  • Symptom Persistence: detected
  • Anxiety Indicators: detected
  • Urgency expressed
  • Duration mentioned
  • Seeking medical attention

---------------------

In [5]:
print("\n--- DataFrame Info ---")
results_df_export.info()

print("\n--- Descriptive Statistics ---")
display(results_df_export.describe())


--- DataFrame Info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Conversation_ID  100 non-null    int64  
 1   HIV_Risk_Score   100 non-null    int64  
 2   HIV_Risk_Level   100 non-null    object 
 3   MH_Risk_Score    100 non-null    int64  
 4   MH_Risk_Level    100 non-null    object 
 5   Priority         100 non-null    float64
dtypes: float64(1), int64(3), object(2)
memory usage: 4.8+ KB

--- Descriptive Statistics ---


,Conversation_ID,HIV_Risk_Score,MH_Risk_Score,Priority
count,100.000000,100.000000,100.0,100.00000
mean,50.500000,69.300000,15.0,0.86000
std,29.011492,9.561603,0.0,0.22563
min,1.000000,50.000000,15.0,0.50000
25%,25.750000,60.000000,15.0,0.50000
50%,50.500000,75.000000,15.0,1.00000
75%,75.250000,75.000000,15.0,1.00000
max,100.000000,75.000000,15.0,1.00000
